In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import os, math, random, io, requests, json, time, torch
from torch import nn
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg as agg

from datasets import load_dataset, Dataset, Image as HFImage
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoImageProcessor,
    AutoModel,
)
from torch.amp import autocast, GradScaler
from concurrent.futures import ThreadPoolExecutor, as_completed


# CONFIGURATION


LLM_NAME    = "meta-llama/Llama-3.2-1B"
VISION_NAME = "facebook/dinov3-vitl16-pretrain-lvd1689m"

cache_dir        = "/content/livis_cache"
max_cached_items = 50_000
train_size       = 46_500
val_size         = 3_000

# S2 Configuration
# Batch Size 4 -> Effectively 20 images per step (1 global + 4 crops per image).
# If you have >40GB VRAM, you can try 6 or 8.
batch_size    = 24
s2_img_size   = 448    # High-res input
base_img_size = 224    # Native DINO resolution

max_txt_len  = 128
total_steps  = 50000
warmup_steps = 1000
grad_accum   = 32     # High accumulation to stabilize the small batch size
val_interval = 500    # Validate every 500 steps
save_step_10k = 10000

use_wandb   = True
wandb_project = "blip2-llama-livis-dinov3-s2"
wandb_val_samples = 4

RESUME = False
RESUME_QFORMER_PATH   = "checkpoints/qformer_best.pt"
RESUME_PROJECTOR_PATH = "checkpoints/projector_best.pt"

device = "cuda"
torch.backends.cuda.matmul.allow_tf32 = True
print("Device:", device)

if torch.cuda.is_available() and torch.cuda.is_bf16_supported():
    print("✅ BFloat16 supported! Using bfloat16 for Vision Encoder.")
    vision_dtype = torch.bfloat16
else:
    print("⚠️ BFloat16 NOT supported. Using float32 for Vision Encoder.")
    vision_dtype = torch.float32

os.makedirs(cache_dir, exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)



# MODEL LOADING


print(f"Loading LLM: {LLM_NAME}")
tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
llm = AutoModelForCausalLM.from_pretrained(LLM_NAME, torch_dtype=torch.float16, device_map="auto")
llm.eval()
d_model = llm.config.hidden_size

print(f"Loading Vision: {VISION_NAME}")
vision_model = AutoModel.from_pretrained(VISION_NAME, torch_dtype=vision_dtype, trust_remote_code=True).to(device)
vision_processor = AutoImageProcessor.from_pretrained(VISION_NAME, trust_remote_code=True)
vision_model.eval()

# Force processor to use our High-Res S2 size
if hasattr(vision_processor, "size"):
    vision_processor.size = {"height": s2_img_size, "width": s2_img_size}
if hasattr(vision_processor, "crop_size"):
    vision_processor.crop_size = {"height": s2_img_size, "width": s2_img_size}

d_vision = vision_model.config.hidden_size
patch_sz = getattr(vision_model.config, "patch_size", 16)
grid_sz  = base_img_size // patch_sz # Usually 224//16 = 14


# S2-WRAPPER IMPLEMENTATION


class S2VisionModule(nn.Module):
    """
    S2-Wrapper: Scale on Scales.
    Processes a high-res image (448) by:
    1. Resizing to base (224) for Global view.
    2. Splitting into 4x 224 crops for Local views.
    3. Concatenating tokens -> 1 Global + 4 Local sets.
    """
    def __init__(self, vision_model, base_size=224):
        super().__init__()
        self.vision_model = vision_model
        self.base_size = base_size
        self.dtype = vision_model.dtype

    def forward(self, pixel_values):
        B, C, H, W = pixel_values.shape

        # 1. Global Scale
        global_view = F.interpolate(pixel_values, size=(self.base_size, self.base_size), mode='bilinear', align_corners=False)

        # 2. Local Scale (Unfold)
        kernel_size = self.base_size
        stride = self.base_size
        patches = pixel_values.unfold(2, kernel_size, stride).unfold(3, kernel_size, stride)
        p_h = patches.size(2)
        p_w = patches.size(3)

        # Reshape to (B * num_crops, C, H_crop, W_crop)
        local_views = patches.permute(0, 2, 3, 1, 4, 5).contiguous().reshape(-1, C, kernel_size, kernel_size)

        # 3. Forward Passes (Batching Global + Local)
        all_imgs = torch.cat([global_view, local_views], dim=0)
        outputs = self.vision_model(pixel_values=all_imgs.to(self.dtype))
        last_hidden = outputs.last_hidden_state

        # Remove CLS token (index 0)
        # We keep everything else (including registers) because we handle them later
        spatial_tokens = last_hidden[:, 1:, :]

        # 4. Reconstruct structure
        global_tokens = spatial_tokens[:B] # (B, N, D)

        local_tokens = spatial_tokens[B:]
        local_tokens = local_tokens.reshape(B, p_h * p_w, -1, d_vision)
        local_tokens_flat = local_tokens.reshape(B, -1, d_vision)

        # 5. Concatenate Global + Local
        # Order: [Global, TopLeft, TopRight, BotLeft, BotRight] (assuming 2x2 grid)
        combined_tokens = torch.cat([global_tokens, local_tokens_flat], dim=1)

        return combined_tokens

s2_vision = S2VisionModule(vision_model, base_size=base_img_size)


# DATASET


print("Loading dataset structure...")
raw = load_dataset("laion/220k-GPT4Vision-captions-from-LIVIS", split="train")

paths_ok = []
caps_ok  = []
jobs     = []
needed   = max_cached_items

print(f"Scanning dataset (target {needed})...")
for idx, ex in enumerate(raw):
    if len(paths_ok) + len(jobs) >= needed: break
    url = ex.get("url")
    if not url: continue
    img_path = os.path.join(cache_dir, f"{idx:06d}.jpg")
    txt_path = os.path.join(cache_dir, f"{idx:06d}.txt")

    sc = (ex.get("short_caption") or "").strip()
    if not sc: sc = (ex.get("caption") or "An image.").strip()
    cap = f"Short caption: {sc}"

    if os.path.exists(img_path) and os.path.getsize(img_path) > 100:
        paths_ok.append(img_path)
        caps_ok.append(cap)
        if not os.path.exists(txt_path):
            with open(txt_path, "w", encoding="utf-8") as f: f.write(cap)
    else:
        jobs.append((idx, url, cap))

print(f"Found {len(paths_ok)} cached items. Queued {len(jobs)} downloads.")

def fetch_one(job):
    idx, url, cap = job
    img_path = os.path.join(cache_dir, f"{idx:06d}.jpg")
    txt_path = os.path.join(cache_dir, f"{idx:06d}.txt")
    try:
        r = requests.get(url, timeout=5)
        img = Image.open(io.BytesIO(r.content)).convert("RGB")
        img.save(img_path, "JPEG", quality=90)
        with open(txt_path, "w", encoding="utf-8") as f: f.write(cap)
        return (img_path, cap)
    except: return None

if jobs:
    print(f"🚀 Downloading {len(jobs)} images...")
    with ThreadPoolExecutor(max_workers=40) as ex_pool:
        futures = {ex_pool.submit(fetch_one, j): j for j in jobs}
        for i, fut in enumerate(as_completed(futures)):
            res = fut.result()
            if res:
                p, c = res
                paths_ok.append(p)
                caps_ok.append(c)
            if i % 1000 == 0 and i > 0: print(f"Progress: {i}/{len(jobs)}...")

if len(paths_ok) < (train_size + val_size):
    print("⚠️ Resizing splits.")
    val_size = min(val_size, int(len(paths_ok) * 0.1))
    train_size = len(paths_ok) - val_size

hf_ds = Dataset.from_dict({"image": paths_ok, "caption": caps_ok}).cast_column("image", HFImage()).shuffle(seed=42)
train_ds = hf_ds.select(range(train_size))
val_ds   = hf_ds.select(range(train_size, train_size + val_size))

def collate_fn(batch):
    images, caps = [], []
    for ex in batch:
        img = ex["image"]
        if not isinstance(img, Image.Image):
            try: img = img.convert("RGB")
            except: continue
        else: img = img.convert("RGB")
        images.append(img)
        caps.append(ex["caption"])
    if not images: return None
    pixel_values = vision_processor(images=images, return_tensors="pt")["pixel_values"]
    enc = tokenizer(caps, padding="max_length", truncation=True, max_length=max_txt_len, return_tensors="pt")
    return {"pixel_values": pixel_values, "input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "captions": caps}

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, collate_fn=collate_fn)



# ARCHITECTURE (With View Embeddings)


class QFormerBlock(nn.Module):
    def __init__(self, d_model, n_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, batch_first=True, dropout=dropout)
        self.ln2 = nn.LayerNorm(d_model)
        self.cross_attn = nn.MultiheadAttention(d_model, n_heads, batch_first=True, dropout=dropout)
        self.ln3 = nn.LayerNorm(d_model)
        hidden = int(d_model * mlp_ratio)
        self.mlp = nn.Sequential(nn.Linear(d_model, hidden), nn.GELU(), nn.Linear(hidden, d_model), nn.Dropout(dropout))

    def forward(self, q, v):
        q2, _ = self.self_attn(self.ln1(q), self.ln1(q), self.ln1(q), need_weights=False)
        q = q + q2
        q2, attn = self.cross_attn(self.ln2(q), self.ln2(v), self.ln2(v), need_weights=True, average_attn_weights=False)
        q = q + q2
        q = q + self.mlp(self.ln3(q))
        return q, attn

class QFormer(nn.Module):
    def __init__(self, d_vis, d_model, n_queries=32, n_layers=8, heads=8):
        super().__init__()
        self.query = nn.Parameter(torch.randn(n_queries, d_model))
        self.vis_proj = nn.Linear(d_vis, d_model)

        # --- NEW: View Embeddings for S2 ---
        # 5 views: [Global, TL, TR, BL, BR]
        # This helps the model distinguish spatial relationships between crops
        self.view_emb = nn.Parameter(torch.randn(5, 1, d_model))
        # -----------------------------------

        self.layers = nn.ModuleList([QFormerBlock(d_model, heads) for _ in range(n_layers)])
        self.final_ln = nn.LayerNorm(d_model)
        self.last_attn = None

    def forward(self, vis_tokens):
        # vis_tokens: (B, Total_Tokens, D_vis)
        v = self.vis_proj(vis_tokens)
        B, N, D = v.shape

        # Add View Embeddings
        num_views = 5
        if N % num_views == 0:
            tokens_per_view = N // num_views
            v = v.view(B, num_views, tokens_per_view, D)
            # Broadcast (5, 1, D) -> (B, 5, T, D)
            v = v + self.view_emb.unsqueeze(0)
            v = v.reshape(B, N, D)

        q = self.query.unsqueeze(0).expand(B, -1, -1)
        for blk in self.layers:
            q, attn = blk(q, v)
            self.last_attn = attn
        return self.final_ln(q)

class Projector(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(nn.Linear(d_model, d_model*4), nn.GELU(), nn.Linear(d_model*4, d_model))
        self.ln2 = nn.LayerNorm(d_model)
    def forward(self, x):
        return self.ln2(x + self.mlp(self.ln1(x)))

class BLIP2(nn.Module):
    def __init__(self, llm, s2_vision, qformer, projector):
        super().__init__()
        self.llm, self.s2_vision, self.qformer, self.projector = llm, s2_vision, qformer, projector

    def forward(self, input_ids, pixel_values, attention_mask):
        with torch.no_grad():
            with torch.amp.autocast("cuda", enabled=False):
                pv = pixel_values.to(self.s2_vision.dtype)
                vtoks = self.s2_vision(pixel_values=pv)

        q = self.qformer(vtoks.to(torch.float32))
        q = self.projector(q).to(self.llm.dtype)

        txt_emb = self.llm.get_input_embeddings()(input_ids)
        all_emb = torch.cat([q, txt_emb], dim=1)
        K = q.size(1)

        prefix_mask = torch.ones((input_ids.size(0), K), device=device, dtype=attention_mask.dtype)
        full_mask = torch.cat([prefix_mask, attention_mask], dim=1)

        labels = input_ids.clone()
        labels[attention_mask == 0] = -100
        prefix_labels = torch.full((input_ids.size(0), K), -100, device=device, dtype=torch.long)
        full_labels = torch.cat([prefix_labels, labels], dim=1)

        return self.llm(inputs_embeds=all_emb, attention_mask=full_mask, labels=full_labels)

qformer = QFormer(d_vision, d_model).to(device)
projector = Projector(d_model).to(device)
model = BLIP2(llm, s2_vision, qformer, projector)

for p in llm.parameters(): p.requires_grad = False
for p in vision_model.parameters(): p.requires_grad = False

if RESUME:
    try:
        qformer.load_state_dict(torch.load(RESUME_QFORMER_PATH, map_location=device))
        projector.load_state_dict(torch.load(RESUME_PROJECTOR_PATH, map_location=device))
        print("✅ Resumed from checkpoints.")
    except: print("⚠️ Resume failed, starting fresh.")

train_params = list(qformer.parameters()) + list(projector.parameters())
optimizer = torch.optim.AdamW(train_params, lr=5e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda s: s/warmup_steps if s<warmup_steps else max(0.1, 0.5*(1+math.cos(math.pi*(s-warmup_steps)/(total_steps-warmup_steps)))))
scaler = GradScaler()

if use_wandb:
    import wandb
    if wandb.run is None:
        wandb.init(project=wandb_project, config={"vision": VISION_NAME, "method": "S2-Wrapper-ViewEmb"})



# VISUALIZATION (Corrected)


@torch.no_grad()
def generate_caption(img: Image.Image):
    img = img.convert("RGB")
    pv = vision_processor(images=img, return_tensors="pt")["pixel_values"].to(device).to(vision_dtype)
    vtoks = s2_vision(pixel_values=pv)
    q = qformer(vtoks.to(torch.float32))
    q = projector(q).to(llm.dtype)

    prompt = "Short caption: "
    ids = tokenizer(prompt, return_tensors="pt").to(device)["input_ids"]
    txt_emb = llm.get_input_embeddings()(ids)
    all_emb = torch.cat([q, txt_emb], dim=1)

    out = llm.generate(inputs_embeds=all_emb, max_new_tokens=32, do_sample=False)
    return tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()

def get_heatmap_image(img, display_count=8):
    """
    Robust heatmap generation for S2 + DINOv3 (handles registers automatically).
    """
    img = img.convert("RGB")
    pv = vision_processor(images=img, return_tensors="pt")["pixel_values"].to(device).to(vision_dtype)

    with torch.no_grad():
        vtoks = s2_vision(pixel_values=pv)
        qformer(vtoks.to(torch.float32))
        attn_all = qformer.last_attn # (Batch=1, Heads, Queries, Keys)

        # 1. Average over heads
        attn_avg = attn_all.mean(dim=1).squeeze(0) # (32, TotalTokens)

        # 2. Logic to extract the Spatial Grid from Global View
        total_tokens = attn_avg.size(1)
        num_views = 5
        tokens_per_view = total_tokens // num_views # e.g. 200

        expected_patches = grid_sz * grid_sz # e.g. 196
        num_registers = tokens_per_view - expected_patches # e.g. 4

        # We only look at Global View (first chunk), avoiding registers
        # attn_global_patches: (32, 196)
        attn_global_patches = attn_avg[:, num_registers : tokens_per_view]


    # Plotting

    display_count = min(display_count, 32)
    total_cols = display_count + 1
    fig, axes = plt.subplots(1, total_cols, figsize=(3 * total_cols, 3))

    axes[0].imshow(img)
    axes[0].set_title("Input")
    axes[0].axis("off")

    img_np = np.array(img)

    for i in range(display_count):
        ax = axes[i+1]

        # Grab specific query attention
        heat = attn_global_patches[i].view(grid_sz, grid_sz).float().cpu().numpy()
        heat = (heat - heat.min()) / (heat.max() - heat.min() + 1e-8)

        heat_img = Image.fromarray((heat * 255).astype(np.uint8)).resize(img.size, resample=Image.BILINEAR)

        ax.imshow(img_np)
        ax.imshow(heat_img, cmap="jet", alpha=0.55)
        ax.set_title(f"Q{i}")
        ax.axis("off")

    plt.tight_layout()
    canvas = agg.FigureCanvasAgg(fig)
    canvas.draw()
    rgba = np.asarray(canvas.buffer_rgba())
    pil_img = Image.fromarray(rgba)
    plt.close(fig)

    return pil_img.convert("RGB")


# TRAINING LOOP

print("🚀 Training starting...")
best_val_loss = float("inf")
global_step = 0
train_iter = iter(train_loader)
running_loss = 0.0

while global_step < total_steps:
    try: batch = next(train_iter)
    except:
        train_iter = iter(train_loader)
        batch = next(train_iter)
    if not batch: continue

    pixel_values = batch["pixel_values"].to(device)
    input_ids    = batch["input_ids"].to(device)
    attn_mask    = batch["attention_mask"].to(device)

    with autocast("cuda", dtype=torch.float16):
        out = model(input_ids, pixel_values, attn_mask)
        loss = out.loss / grad_accum

    scaler.scale(loss).backward()
    running_loss += loss.item()

    if (global_step + 1) % grad_accum == 0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(train_params, 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad()

    if global_step % 200 == 0:
        avg = running_loss / max(1, global_step+1)
        lr = scheduler.get_last_lr()[0]
        print(f"[{global_step}] loss={loss.item()*grad_accum:.4f} avg={avg*grad_accum:.4f} lr={lr:.6e}")
        if use_wandb: wandb.log({"train_loss": loss.item()*grad_accum, "lr": lr}, step=global_step)

    if (global_step + 1) % val_interval == 0:
        model.eval()
        v_loss = 0
        count = 0
        with torch.no_grad():
            for vb in val_loader:
                if not vb: continue
                with autocast("cuda", dtype=torch.float16):
                    out = model(vb["input_ids"].to(device), vb["pixel_values"].to(device), vb["attention_mask"].to(device))
                v_loss += out.loss.item()
                count += 1
                if count >= 100: break
        v_loss /= max(1, count)
        print(f"VAL Loss: {v_loss:.4f}")

        if use_wandb:
            wandb.log({"val_loss": v_loss}, step=global_step)
            columns = ["step", "image_vis", "gt", "pred"]
            val_table = wandb.Table(columns=columns)
            indices = random.sample(range(len(val_ds)), min(len(val_ds), wandb_val_samples))
            print(f"Generating visualizations...")
            for idx in indices:
                ex_img = val_ds[idx]["image"]
                ex_gt  = val_ds[idx]["caption"]
                ex_pred = generate_caption(ex_img)
                vis_img = get_heatmap_image(ex_img)
                val_table.add_data(global_step, wandb.Image(vis_img), ex_gt, ex_pred)
            wandb.log({"validation_samples": val_table}, step=global_step)

        if v_loss < best_val_loss:
            best_val_loss = v_loss
            torch.save(qformer.state_dict(), "checkpoints/qformer_best.pt")
            torch.save(projector.state_dict(), "checkpoints/projector_best.pt")

        model.train()

    global_step += 1

Device: cuda
✅ BFloat16 supported! Using bfloat16 for Vision Encoder.
Loading LLM: meta-llama/Llama-3.2-1B


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading Vision: facebook/dinov3-vitl16-pretrain-lvd1689m
Loading dataset structure...
Scanning dataset (target 50000)...
Found 50000 cached items. Queued 0 downloads.


wandb: Currently logged in as: eren23 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


🚀 Training starting...
[0] loss=8.0751 avg=8.0751 lr=0.000000e+00
[200] loss=7.1019 avg=7.6656 lr=3.000000e-07
[400] loss=4.7358 avg=6.8057 lr=6.000000e-07


...

[24600] loss=2.1059 avg=2.4617 lr=3.840000e-05
[24800] loss=2.0879 avg=2.4593 lr=3.875000e-05


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


VAL Loss: 2.1689
Generating visualizations...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[25000] loss=2.0957 avg=2.4568 lr=3.905000e-05
[25200] loss=2.1102 avg=2.4544 lr=3.935000e-05
